<a href="https://colab.research.google.com/github/Pro5507/ABS-and-Polly/blob/main/Image_To_Text_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from config import HF_API_KEY
import requests
from PIL import Image
import io
import os
from colorama import init, Fore, Style
import json

init(autoreset=True)
def query_hf_api(api_url, payload=None, files=None, method="post"):
  headers = {"Authorization": f"Bearer {HF_API_KEY}"}
  try:
    if method.lower() == "post":
      response = requests.post(api_url, headers=headers, json=payload, files=files)
    else:
      response = requests.get(api_url, headers=headers, params=payload)
      if response.status_code != 200:
        raise Exception(f"Status {response.status_code}: {response.text}")
      return response.content
  except Exception as e:
    print(f"{Fore.RED}× Error while calling API: {e}")
    raise

def get_basic_caption(image, model="nlpconnect/vit-gpt-2-image-captioning"):
  print(f"{Fore.YELLOW}???? Generating basic caption using vit-gpt2-image-captioning...")
  api_url = f"https://api-inference.huggingface.co/models/{model}"
  buffered = io.BytesIO()
  image.save(buffered, format="JPEG")
  buffered.seek(0)
  headers = {"Authorization": f"Bearer {HF_API_KEY}"}
  response = requests.post(api_url, headers=headers, data=buffered.read())
  result = response.json()
  if isinstance(result, dict) and "error" in result:
    return f"Error {result['error']}"
  caption = result[0].get("generated_text","No caption generated")
  return caption

def generate_text(prompt, model="gpt2", max_new_tokens=60):
  print(f"{Fore.CYAN}????Generating text with prompt: {prompt}")
  api_url = f"https://api-inference.huggingface.co/models/{model}"
  payload = {"inputs": prompt, "parameters": {"max_new_tokens": max_new_tokens}}
  text_bytes = query_hf_api(api_url, payload)
  try:
    result = json.loads(text_bytes)